## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Descubrimiento de grupos en los datos

---

En esta libreta se generan los modelos de clusterización sobre el dataset final del escenario para su posterior evaluación y análisis. Se utilizará como herramienta de soporte a mlflow para el registro completo de la experimentación. 

---

In [1]:
# Importación de librerías

import pandas as pd
# Se importa mlflow para registro de la experimentación 
import mlflow
# Se importan los métodos a utilizar para clusterizar
from sklearn.cluster import KMeans, AgglomerativeClustering, Birch
# Otras opciones: DBSCAN, MeanShift, BisectingKMeans

# Se importan las métricas a utilizar para evaluar el proceso
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from mlflow.models.signature import infer_signature

----

##### Lectura del dataset

In [2]:
df = pd.read_csv("../../../data/final/datos_finales_clusterizacion.csv", sep=";")
df.head(5)

,edad,importe_solicitado,duracion_credito,antiguedad_empleado,ingresos,pct_ingreso,tasa_interes,estado_credito,antiguedad_cliente,gastos_ult_12m,...,estado_cliente_ACTIVO,estado_cliente_PASIVO,genero_F,genero_M,nivel_educativo_DESCONOCIDO,nivel_educativo_POSGRADO_COMPLETO,nivel_educativo_POSGRADO_INCOMPLETO,nivel_educativo_SECUNDARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_INCOMPLETO
0,21,1000,2,5.0,9600,0.10,11.14,0,39.0,1144.0,...,True,False,False,True,False,False,False,True,False,False
1,23,35000,2,4.0,65500,0.53,15.23,1,36.0,1887.0,...,True,False,False,True,False,False,False,False,True,False
2,24,35000,4,8.0,54400,0.55,14.27,1,54.0,1314.0,...,True,False,False,True,True,False,False,False,False,False
3,21,2500,2,2.0,9900,0.25,7.14,1,34.0,1171.0,...,True,False,True,False,False,False,False,True,False,False
4,26,35000,3,8.0,77100,0.45,12.42,1,21.0,816.0,...,True,False,False,True,False,False,False,False,False,True


In [3]:
# Transformación por las características de mlflow para la detección de nulos en campos de tipo integer
df = df.astype({col: 'float64' for col in df.select_dtypes(include='int').columns})

----

##### Configuración de la experimentación

In [4]:
# Se genera el experimento en mlflow
exp_name = 'Experimentación clustering #1 (SP7)'
exp_id = mlflow.create_experiment(name=exp_name)

In [5]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="Kmeans - K=2"):
    # Selección y configuración de la técnica
    modelo_clusters = KMeans(n_clusters=2)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 2)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    input_example = df.iloc[0].to_frame().T
    signature = infer_signature(df, cluster_labels)
    mlflow.sklearn.log_model(trained_model, "Kmeans_K2", input_example=input_example, signature=signature)

    # Se finaliza el registro
    mlflow.end_run()

2025/05/06 19:58:58 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Para contabilizar los elementos por cluster se utilizan las labels generadas

In [6]:
# Se preserva el conjunto de datos original para evitar problemas
df_kmeans_k2 = df.copy()
# Se agregan las labels generadas
df_kmeans_k2['cluster'] = cluster_labels
df_kmeans_k2['cluster'].value_counts()

cluster
0    8276
1     602
Name: count, dtype: int64

Se hace la exportación de los resultados a un archivo .csv nuevo

In [7]:
df_kmeans_k2.to_csv("../../../data/final/df_kms_k2.csv", sep=";", index=False)

Se pasa a probar otro método (clustering jerárquico)

In [8]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="AGClustering - K=2"):
    # Selección y configuración de la técnica
    modelo_clusters = AgglomerativeClustering(n_clusters=2)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 2)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    # input_example = df.iloc[0].to_frame().T
    signature = infer_signature(df, cluster_labels)
    mlflow.sklearn.log_model(trained_model, "AGC_2", signature=signature)

    # Se finaliza el registro
    mlflow.end_run()

2025/05/06 20:00:00 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/05/06 20:00:08 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Para contabilizar los elementos por cluster se utilizan las labels generadas

In [9]:
# Se preserva el conjunto de datos original para evitar problemas
df_agc_k2 = df.copy()
# Se agregan las labels generadas
df_agc_k2['cluster'] = cluster_labels
df_agc_k2['cluster'].value_counts()

cluster
0    8508
1     370
Name: count, dtype: int64

Se realiza la exportación de los resultados

In [10]:
df_agc_k2.to_csv("../../../data/final/df_agc_k2.csv", sep=";", index=False)

Prueba con otro método paramétrico

In [11]:
with mlflow.start_run(experiment_id=exp_id, run_name="Birch - K = 2"):
    modelo_clusters = Birch(n_clusters=2)
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Calcular métricas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 2)

    # Registrar métricas
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Guardar el modelo
    signature = infer_signature(df, cluster_labels)
    mlflow.sklearn.log_model(trained_model, "Birch_Model", signature=signature)

    mlflow.end_run()

2025/05/06 20:03:48 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


In [12]:
# Se preserva el conjunto de datos original para evitar problemas
df_bc = df.copy()
# Se agregan las labels generadas
df_bc['cluster'] = cluster_labels
df_bc['cluster'].value_counts()

cluster
0    8508
1     370
Name: count, dtype: int64

Se prueba con Kmeans pero con K = 3

Para contabilizar los elementos por cluster se utilizan las labels generadas

In [13]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="Kmeans - K=3"):
    # Selección y configuración de la técnica
    modelo_clusters = KMeans(n_clusters=3)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 3)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    input_example = df.iloc[0].to_frame().T
    signature = infer_signature(df, cluster_labels)
    mlflow.sklearn.log_model(trained_model, "Kmeans_K3", input_example=input_example, signature=signature)

    # Se finaliza el registro
    mlflow.end_run()

2025/05/06 20:25:55 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


In [14]:
# Se preserva el conjunto de datos original para evitar problemas
df_kms_k3 = df.copy()
# Se agregan las labels generadas
df_kms_k3['cluster'] = cluster_labels
df_kms_k3['cluster'].value_counts()

cluster
0    4701
1    3872
2     305
Name: count, dtype: int64

### Opciones de optimización del proceso

Utilizar PCA (solo para visualización de clusters) y un proceso de normalización de valores para eliminar el impacto de las diferencias en el dataset en la clusterización.

In [15]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Normalización de los datos
scaler = StandardScaler()
df_normalizado = scaler.fit_transform(df)

# Aplicación de PCA
pca = PCA(n_components=2)
df_pca = pca.fit_transform(df_normalizado)

# Clusterización con las diferentes técnicas

In [16]:
# Se genera el experimento en mlflow
exp_name = 'Experimentación clustering #2'
exp_id = mlflow.create_experiment(name=exp_name)

In [17]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="Kmeans N - K=2"):
    # Selección y configuración de la técnica
    modelo_clusters = KMeans(n_clusters=2)
    
    trained_model = modelo_clusters.fit(df_normalizado)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df_normalizado, cluster_labels)
    score_db = davies_bouldin_score(df_normalizado, cluster_labels)
    score_ch = calinski_harabasz_score(df_normalizado, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 2)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    # input_example = df_normalizado.iloc[0].to_frame().T
    signature = infer_signature(df_normalizado, cluster_labels)
    mlflow.sklearn.log_model(trained_model, "Kmeans_K2_N", input_example=input_example, signature=signature)

    # Se finaliza el registro
    mlflow.end_run()

2025/05/06 20:35:22 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


In [18]:
# Se preserva el conjunto de datos original para evitar problemas
# Convertir df_normalizado a un DataFrame de pandas
df_kms_k2_n = pd.DataFrame(df_normalizado, columns=df.columns)

# Se agregan las labels generadas
df_kms_k2_n['cluster'] = cluster_labels

# Contar los elementos por cluster
df_kms_k2_n['cluster'].value_counts()

cluster
1    6850
0    2028
Name: count, dtype: int64